In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import imageio as iio
import deeplabcut
import subprocess
import datetime
from tqdm import tqdm
import cv2

Loading DLC 3.0.0rc6...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
def read_frames_chunked(filename, output_file, chunk_size=100, fps=30, pixel_format="gray16le", movie_dtype="uint16", frame_size=(640, 576), mapping="DEPTH", finfo=None, **kwargs):
    # if finfo is None:
        # finfo = get_video_info(filename, **kwargs)
    
    # if not frame_size:
        # frame_size = finfo["dims"]
    
    # total_frames = finfo["nframes"]
    total_frames = 1000
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(str(output_file), fourcc, fps, (frame_size[0], frame_size[1]), isColor=False)
    
    for start in range(0, total_frames, chunk_size):
        end = min(start + chunk_size, total_frames)
        command = [
            "ffmpeg", "-loglevel", "fatal", "-ss", str(datetime.timedelta(seconds=start / fps)), "-i", filename,
            "-vframes", str(end - start), "-f", "image2pipe", "-s", f"{frame_size[0]}x{frame_size[1]}",
            "-pix_fmt", pixel_format, "-vcodec", "rawvideo", "-"
        ]
        
        # if isinstance(mapping, str):
        #     mapping_dict = get_stream_names(filename)
        #     mapping = mapping_dict.get(mapping, 0)
        
        # if filename.endswith((".mkv", ".avi")):
        #     command += ["-map", f"0:{mapping}", "-vsync", "0"]
        
        pipe = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = pipe.communicate()
        
        if err:
            print("Error:", err)
            continue
        
        video_chunk = np.frombuffer(out, dtype=movie_dtype).reshape((end - start, frame_size[1], frame_size[0]))
        
        # Clipping and normalization
        video_chunk = np.clip(video_chunk, 0, 1000)
        video_chunk = (video_chunk / video_chunk.max() * 255).astype(np.uint8)
        
        for frame in video_chunk:
            video_writer.write(frame)
    
    video_writer.release()

def process_video(path, chunk_size=10, dlc_config='/n/groups/datta/jlove/data/rat_seq/rat_seq_paper/keypoint_model/config-v2.yaml'):
    print('Processing video in chunks')
    output_file = path.parent / 'ir_clipped.avi'
    read_frames_chunked(path.as_posix(), output_file, chunk_size=chunk_size)
    
    videos = [output_file.as_posix()]
    deeplabcut.analyze_videos(
        dlc_config, videos, videotype='avi', shuffle=1, trainingsetindex=0, gputouse=0, save_as_csv=True, destfolder=None, dynamic=(True, .5, 10)
    )
    deeplabcut.filterpredictions(dlc_config, videos)
    deeplabcut.create_labeled_video(dlc_config, videos, filtered=True, pcutoff=.3)


In [3]:
folder = Path('/n/groups/datta/jlove/data/rat_seq/lesion')
irs = list(folder.glob('**/ir.avi'))

In [4]:
from tqdm import tqdm

In [5]:
irs = [Path('/n/groups/datta/jlove/data/rat_seq/lesion/data/session_20250311154526/ir.avi')]

In [6]:
irs = [Path('/n/groups/datta/jlove/data/tst5.avi')]

In [8]:
for ir in tqdm(irs, total=len(irs)):
    process_video(ir)

  0%|                                                                                                                                                                                        | 0/1 [00:00<?, ?it/s]

Processing video in chunks
Using snapshot-50000 for model /n/groups/datta/jlove/data/rat_seq/rat_seq_paper/keypoint_model/dlc-models/iteration-0/KeypointMoSeqDLCOct18-trainset95shuffle1
Starting analysis in dynamic cropping mode with parameters: (True, 0.5, 10)
Switching batchsize to 1, num_outputs (per animal) to 1 and TFGPUinference to False (all these features are not supported in this mode).


/home/jal5475/.miniconda/envs/DEEPLABCUT/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2025-03-14 13:02:30.147801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43598 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:ca:00.0, compute capability: 8.6


Starting to analyze %  /n/groups/datta/jlove/data/ir_clipped.avi
Loading  /n/groups/datta/jlove/data/ir_clipped.avi
Duration of video [s]:  33.33 , recorded with  30.0 fps!
Overall # of frames:  1000  found with (before cropping) frame dimensions:  640 576
Starting to extract posture



2025-03-14 13:02:32.109342: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101                                                                               | 0/1000 [00:00<?, ?it/s]
2025-03-14 13:02:34.167304: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.

%|█▋                                                                                                                                                                          | 10/1000 [00:04<07:03,  2.34it/s]
%|███▍                                                                                                                                                                        | 20/1000 [00:04<03:11,  5.13it/s]
%|█████▏                                                                                                                                                                      | 30/1000 [00:04<01:55,  8

Saving results in /n/groups/datta/jlove/data...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Filtering with median model /n/groups/datta/jlove/data/ir_clipped.avi
Saving filtered csv poses!
Starting to process video: /n/groups/datta/jlove/data/ir_clipped.avi
Loading /n/groups/datta/jlove/data/ir_clipped.avi and data.
Duration of video [s]: 33.33, recorded with 30.0 fps!
Overall # of frames: 1000 with cropped frame dimensions: 640 576
Generating frames and creating video.



%|                                                                                                                                                                                     | 0/1000 [00:00<?, ?it/s]
%|█████▍                                                                                                                                                                     | 32/1000 [00:00<00:03, 316.23it/s]
%|███████████▎                                                                                                                                                               | 66/1000 [00:00<00:02, 328.41it/s]
%|█████████████████                                                                                                                                                         | 100/1000 [00:00<00:02, 331.49it/s]
%|██████████████████████▊                                                                                                                                          